-sandbox
<img width = '500' src="https://databricks.com/wp-content/uploads/2021/06/db-pride-logo.svg" style="float: left: margin: 30px"/>        <img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 10px"/>              

<br>
> # Bronze para Silver

<br>
## 07 - Cliques

- Os investidores estão clicando em vários tickers para ver os gráficos de série temporal durante a sessão.              
- Registramos essas informações no campo de itens clicados. 

Tabalhar na versão limpa das transações de ações e descobrir o tempo gasto nos tickers.

-sandbox
<h2 style="color:red">Instruções:</h2>

- o que estamos construindo agora: clicklog detalhado, extraindo informações aninhadas da tabela Silver
- por que é útil: esta extração simplifica a geração da tabela Gold

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Configurações úteis<br>

In [0]:
%run ./Includes/13-ClassroomSetup

Preparing the Python environment...

Defining courseware-specific utility methods...

Defining user-facing utility methods...

Defining custom variables for this lesson...

Initializing Databricks Academy's testing framework...

...

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Cleaning up the learning environment... Removed working directory: dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/07_cliques_psp

WARNING: This notebook was tested on DBR 7.0, but we found DBR 7.3. 
 Using an untested DBR may yield unexpected results and/or various errors 
 Please update your cluster configuration and/or download a newer version of this course before proceeding.

The following variables were defined for you: 
 financeDataPath : dbfs:/mnt/training/finance-org 
 The location of the finance dataset used in this capstone project. 
 
 sourceDataPath : dbfs:/mnt/training/finance-org/solutions 
 The location of the specific datasets used in this capstone project as source data. 
 
 silverStockOrdersPath : dbfs:/mnt/training/finance-org/solutions/silver/stock_orders 
 dbfs:/mnt/training/finance-org/solutions/silver/stock_orders 
 
 targetDirectory : dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/07_cliques_psp/silver/clicked_tickers 
 dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/07_cliques_psp/silver/clicked_tickers 
 The following database were created for you: 
 Now using the database identified by databaseName : 
 thomaz_thomazrossito_com_br_capstone_finance_07_cliques_psp 
 This is a private, per-notebook, database used to provide isolation from other users and exercises. 
 All done!

Declared the following function:
 realityCheck

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Etapas a serem concluídas:

<br>

- Ler a tabela de Pedidos de Estoque a partir do diretório **`silverStockOrdersPath`** para um DataFrame
- Selecionar apenas as colunas **`transaction_id`**, **`investor`**, **`dow`**, **`clicked_items`**
- Crie UDF para calcular o tempo total gasto para **`clicked_items`**
- Crie uma nova coluna chamada **`time_spent_on_site`** usando a UDF que acabou de criar
- Transforme o DataFrame para ter **`clicked_items`** por linha e renomeie como **`ticker_clicked`**. "Deve conter apenas o nome do ticker"
- crie uma coluna **`time_spent_on_ticker`** para ter o tempo gasto naquele único ticker
- Grave o DataFrame resultante no diretório **`targetDirectory`** como uma tabela Delta usando o modo de substituição.
- Retorne o DataFrame resultante.
  
  
O esquema do DataFrame deve ser:

|name|type|
|---|---|
|transaction_id|LongType|
|investor|LongType|
|dow|IntegerType|
|time_spent_on_site|IntegerType|
|ticker_clicked|StringType|
|time_spent_on_ticker|IntegerType|

In [0]:
def func_cliques(spark, silverStockOrdersPath, targetDirectory):
  
  
  from pyspark.sql.functions import col, explode
  from pyspark.sql.types import IntegerType
  
  
  # Crie UDF para calcular o tempo total gasto para clicked_items
  def time_spent(l):
    times = [int(c[1]) for c in l]
    return __builtins__.sum(times)

  time_spent_udf = udf(time_spent, IntegerType())
  
  
  # Ler a tabela de Pedidos de Estoque a partir do diretório "silverStockOrdersPath" para um DataFrame
  # Selecionar apenas as colunas: "transaction_id", "investor", "dow", "clicked_items"
  # Crie uma nova coluna chamada "time_spent_on_site" usando a UDF que acabou de criar
  # Transforme o DataFrame para ter "clicked_items" por linha e renomeie como "ticker_clicked". "Deve conter apenas o nome do ticker"
  df = (spark.read
             .format("delta")
             .load(silverStockOrdersPath)
             .select("transaction_id", 
                     "investor", 
                     "dow", 
                     "clicked_items")
             .withColumn("time_spent_on_site", time_spent_udf(col("clicked_items")))
             .withColumn("clicked_item", explode("clicked_items"))
             .selectExpr("*", 
                         "clicked_item[0] AS ticker_clicked", 
                         "CAST(clicked_item[1] AS int) AS time_spent_on_ticker")
             .drop("clicked_items", "clicked_item"))

  
  # Escreve o resultado do Dataframe no diretório "targetDirectory" em tabela Delta usnaod o modo de subistituição
  df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .format("delta") \
    .save(targetDirectory)  

  
  # Retorna o final
  return df

In [0]:
finalDF = func_cliques(spark, silverStockOrdersPath, targetDirectory)

In [0]:
display(finalDF)

transaction_id,investor,dow,time_spent_on_site,ticker_clicked,time_spent_on_ticker
581346394,338,1,128,INCY,41
581346394,338,1,128,RHT,39
581346394,338,1,128,TWX,48
581346415,207,1,22,ADS,13
581346415,207,1,22,NOC,9
581346711,754,3,97,TROW,56
581346711,754,3,97,INCY,41
581347261,646,3,54,AMP,54
581347397,62,7,83,WDC,19
581347397,62,7,83,CCI,8


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Validação</h2>

In [0]:
realityCheck(func_cliques, spark, silverStockOrdersPath, targetDirectory)

Points,Test,Result
1,Returns correct schema,
1,Returns DataFrame with correct number of rows,
1,Returns DataFrame with correct results,
1,Delta table in place,
1,Delta table has correct content,


## <img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Próximo <p>

> * [Métricas diarias de estoque]($./08-Metricas diarias estoque)

-sandbox

<a href="http://www.apache.org/">Apache Software Foundation</a> <p>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a>